In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [3]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1441,tt0458352,35000000,326551094,The Devil Wears Prada,Meryl Streep|Anne Hathaway|Emily Blunt|Stanley...,David Frankel,Meet Andy Sachs. A million girls would kill to...,The Devil Wears Prada is about a young journal...,109,Comedy|Drama|Romance,Twentieth Century Fox Film Corporation|Dune En...,6/30/2006,7.0,2006
1836,tt0181865,48000000,207515725,Traffic,Michael Douglas|Benicio del Toro|Jacob Vargas|...,Steven Soderbergh,No one gets away clean,An exploration of the United States of America...,148,Drama,USA Films|Initial Entertainment Group (IEG)|Be...,12/27/2000,6.6,2000
1818,tt0187078,90000000,237202299,Gone In Sixty Seconds,Nicolas Cage|Angelina Jolie|Giovanni Ribisi|De...,Dominic Sena,"Ice Cold, Hot Wired.",Upon learning that he has to come out of retir...,118,Action|Crime|Thriller,Jerry Bruckheimer Films|Touchstone Pictures,6/9/2000,6.0,2000
1673,tt0477348,25000000,171600000,No Country for Old Men,Tommy Lee Jones|Javier Bardem|Josh Brolin|Wood...,Joel Coen|Ethan Coen,There are no clean getaways.,"Llewelyn Moss stumbles upon dead bodies, $2 mi...",122,Crime|Drama|Thriller,Miramax Films|Scott Rudin Productions|Paramoun...,11/8/2007,7.6,2007
778,tt1657507,40000000,60965854,Colombiana,Zoe Saldana|Cliff Curtis|Callum Blue|Michael V...,Olivier Megaton,Revenge is beautiful.,"Zoe Saldana plays a young woman who, after wit...",108,Action|Thriller|Crime|Drama,TF1 Films Production|Canal+|CinÃ©CinÃ©ma|Grive...,7/27/2011,6.5,2011


In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

data['profit'] = data['revenue'] - data['budget']
#the time given in the dataset is in string format.
#So we need to change this in datetime format

#Получение словаря с количеством фильмов по жанрам из исходной таблицы
def counter_dict(data, dict, field):
    for subdata in data[field]:
        for thing in subdata.split('|'):
            if thing in dict:
                dict[thing] += 1
            else: dict[thing] = 1
    return dict

data

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015,228436354
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015,185238201
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015,1868178225
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015,1316249360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,tt0120903,75000000,157299717,X-Men,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,Adventure|Action|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,7/13/2000,6.6,2000,82299717
1885,tt0192255,22000000,13555988,The Little Vampire,Richard E. Grant|Jonathan Lipnicki|Jim Carter|...,Uli Edel,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,Horror|Family|Foreign,New Line Cinema,10/27/2000,6.4,2000,-8444012
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,Rene Russo|Jason Alexander|Piper Perabo|Randy ...,Des McAnuff,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,Adventure|Animation|Action|Comedy|Family,Universal Pictures|Capella International|KC Me...,6/30/2000,4.0,2000,-40865180
1887,tt0162983,40000000,36037909,Hanging Up,Meg Ryan|Diane Keaton|Lisa Kudrow|Walter Matth...,Diane Keaton,Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,Comedy|Drama,Laurence Mark Productions|Columbia Pictures Co...,2/16/2000,5.2,2000,-3962091


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"
# +

In [7]:
# тут пишем ваш код для решения данного вопроса:
data.loc[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011,641683000


ВАРИАНТ 2

In [8]:
# можно добавлять разные варианты решения
data.loc[data.budget == data.budget.max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)'
# +

In [10]:
data.loc[data.runtime == data.runtime.max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [11]:
answers['3'] = 'Winnie the Pooh (tt1449283)'
# + 
data.loc[data.runtime == data.runtime.min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [12]:
answers['4'] = '110'
# +
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [13]:
answers['5'] = '107'
# +
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [14]:
# лучше код получения столбца profit вынести в Предобработку что в начале
answers['6'] = 'Avatar (tt0499549)'
#+
data.loc[data.profit == data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,12/10/2009,7.1,2009,2544505847


# 7. Какой фильм самый убыточный? 

In [15]:
answers['7'] = 'The Lone Ranger (tt1210819)'
#+
data.loc[data.profit == data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [16]:
answers['8'] = '1478'
#+
data[data.revenue > data.budget].count()

imdb_id                 1478
budget                  1478
revenue                 1478
original_title          1478
cast                    1478
director                1478
tagline                 1478
overview                1478
runtime                 1478
genres                  1478
production_companies    1478
release_date            1478
vote_average            1478
release_year            1478
profit                  1478
dtype: int64

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [17]:
answers['9'] = 'The Dark Knight (tt0468569)'
#+
data[data.release_year == 2008].sort_values(['revenue'], ascending=False)
data.head(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015,228436354
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015,185238201


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [18]:
answers['10'] = 'The Lone Ranger (tt1210819)'
#+
data2 = data[(data.release_year >= 2012) & (data.release_year <= 2014)].sort_values(['profit'])
data2.head(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090
1214,tt0790736,130000000,61648500,R.I.P.D.,Jeff Bridges|Ryan Reynolds|Kevin Bacon|Stephan...,Robert Schwentke,To protect and serve the living,A recently slain cop joins a team of undead po...,96,Fantasy|Action|Comedy|Crime,Universal Pictures|Original Film|Dark Horse En...,7/18/2013,5.3,2013,-68351500
1007,tt1374992,60000000,8106475,Upside Down,Kirsten Dunst|Jim Sturgess|Timothy Spall|James...,Juan Diego Solanas,Two worlds. One future.,In an alternate universe where twinned worlds ...,104,Romance|Science Fiction|Drama|Fantasy,Onyx Films|Studio 37|Jouror Productions|France...,9/27/2012,6.1,2012,-51893525


# 11. Какого жанра фильмов больше всего?

In [19]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

# Подсмотрено в Slack, интересный метод explode. Лучше копировать в новый дф, чтобы не менялся исходный
data2 = data.copy()

data2['genres'] = data2.genres.str.split('|')
genres = data2.explode('genres')['genres'] # получаем жанры
counter11 = Counter(genres) # создаем счетчик

counter11.most_common(1)

[('Drama', 782)]

ВАРИАНТ 2

In [20]:
answers['11'] = 'Drama'
#+

genres_count_dict = {}

genres_count_dict = counter_dict(data, genres_count_dict, 'genres')
sorted(genres_count_dict.items(), reverse=True, key=lambda x: x[1])  # сортировка по значению словаря

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [21]:
answers['12'] = 'Drama'
#+

genres_count_dict = {}
data_profit = data[data.profit > 0]

genres_count_dict = counter_dict(data_profit, genres_count_dict, 'genres')
sorted(genres_count_dict.items(), reverse=True, key=lambda x: x[1])[0]

('Drama', 560)

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [22]:
answers['13'] = 'Peter Jackson'
#+

data13 = data.copy()
data13['director'] = data13.director.str.split('|')

data13.explode('director').groupby(['director'])['revenue'].sum().sort_values(ascending=False)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 997, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [23]:
answers['14'] = 'Robert Rodriguez'
#+

data14 = data.copy()
data14['director'] = data14.director.str.split('|')

data14 = data14[data14.genres.str.contains('Action', na=False)]
data14.explode('director').groupby(['director'])['imdb_id'].count().sort_values(ascending=False)

director
Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Ridley Scott          6
Antoine Fuqua         6
                     ..
John Whitesell        1
Brian A Miller        1
Jon Amiel             1
Jon Avnet             1
Luis Mandoki          1
Name: imdb_id, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [24]:
answers['15'] = 'Chris Hemsworth'
#+

data15 = data.copy()
data15['cast'] = data15[data15.release_year == 2012].cast.str.split('|')

data15.explode('cast').groupby(['cast'])['revenue'].sum().sort_values(ascending=False)

cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Robert Downey Jr.    1519557910
Mark Ruffalo         1519557910
                        ...    
Michael Nyqvist         3428048
Danny Huston            2106557
Josh Lucas              2106557
Sami Gayle              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [25]:
answers['16'] = 'Matt Damon'
#+

data16 = data.copy()
data16['cast'] = data16[data16.budget >= data16.budget.mean()].cast.str.split('|')

data16.explode('cast').groupby(['cast'])['imdb_id'].count().sort_values(ascending=False)

cast
Matt Damon                 18
Adam Sandler               17
Angelina Jolie             16
Tom Cruise                 15
Eddie Murphy               15
                           ..
Jennifer Hudson             1
Jennifer Jason Leigh        1
Jennifer Saunders           1
Jennifer Tilly              1
Ã“lafur Darri Ã“lafsson     1
Name: imdb_id, Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [26]:
answers['17'] = 'Action'
#+

data17 = data.copy()
data17 = data17[data17.cast.str.contains('Nicolas Cage', na=False)]

genres_count_dict = {}
genres_count_dict = counter_dict(data17, genres_count_dict, 'genres')
sorted(genres_count_dict.items(), reverse=True, key=lambda x: x[1])

[('Action', 17),
 ('Thriller', 15),
 ('Drama', 12),
 ('Crime', 10),
 ('Fantasy', 8),
 ('Adventure', 7),
 ('Comedy', 6),
 ('Science Fiction', 4),
 ('Mystery', 3),
 ('Animation', 3),
 ('Family', 3),
 ('History', 2),
 ('War', 1),
 ('Horror', 1),
 ('Romance', 1)]

# 18. Самый убыточный фильм от Paramount Pictures

In [27]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'
#+

data[data.production_companies.str.contains('Paramount Pictures', na=False)].sort_values('profit')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,7/19/2002,6.0,2002,-64831034
1123,tt0300556,80000000,19480739,Timeline,Paul Walker|Frances O'Connor|Gerard Butler|Bil...,Richard Donner,They had to travel into the past to save the f...,A group of archaeological students become trap...,116,Action|Adventure|Science Fiction,Paramount Pictures|Donners' Company|Mutual Fil...,11/26/2003,5.1,2003,-60519261
1722,tt0435705,70000000,18211013,Next,Nicolas Cage|Julianne Moore|Jessica Biel|Thoma...,Lee Tamahori,"If you can see the future, you can save it.",Las Vegas showroom magician Cris Johnson has a...,96,Action|Science Fiction|Thriller,Paramount Pictures|Virtual Studios|Revolution ...,4/24/2007,5.9,2007,-51788987
1623,tt0375173,60000000,13395939,Alfie,Jude Law|Marisa Tomei|Omar Epps|Jane Krakowski...,Charles Shyer,Meet a man who never met a woman he didn't love.,"In Manhattan, the British limousine driver Alf...",103,Comedy,Paramount Pictures|Patalex Productions,10/22/2004,5.4,2004,-46604061
1631,tt0315297,50000000,25195000,Twisted,Ashley Judd|Samuel L. Jackson|Andy GarcÃ­a|Dav...,Philip Kaufman,Every murder has a mark.,Recently promoted and transferred to the homic...,97,Action|Adventure|Crime|Drama|Thriller,Paramount Pictures|Kopelson Entertainment|Harl...,2/27/2004,5.7,2004,-24805000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,tt1229238,145000000,694713380,Mission: Impossible - Ghost Protocol,Tom Cruise|Jeremy Renner|Simon Pegg|Paula Patt...,Brad Bird,No Plan. No Backup. No Choice.,In the 4th installment of the Mission Impossib...,133,Action|Thriller|Adventure,Paramount Pictures|Skydance Productions|Bad Ro...,12/7/2011,6.7,2011,549713380
1738,tt0418279,150000000,709709780,Transformers,Shia LaBeouf|Josh Duhamel|Megan Fox|Rachael Ta...,Michael Bay,Their war. Our world.,"Young teenager, Sam Witwicky becomes involved ...",144,Adventure|Science Fiction|Action,Paramount Pictures|DreamWorks SKG|Amblin Enter...,6/27/2007,6.6,2007,559709780
603,tt0367882,185000000,786636033,Indiana Jones and the Kingdom of the Crystal S...,Harrison Ford|Cate Blanchett|Shia LaBeouf|Ray ...,Steven Spielberg,The adventure continues . . .,"Set during the Cold War, the Soviets â€“ led b...",122,Adventure|Action,Lucasfilm|Paramount Pictures,5/21/2008,5.6,2008,601636033
317,tt1055369,150000000,836297228,Transformers: Revenge of the Fallen,Shia LaBeouf|Megan Fox|Josh Duhamel|Rainn Wils...,Michael Bay,Revenge is coming,Sam Witwicky leaves the Autobots behind for a ...,150,Science Fiction|Action|Adventure,Paramount Pictures|DreamWorks SKG|Amblin Enter...,6/19/2009,6.0,2009,686297228


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [28]:
answers['19'] = '2015'
#+

data.groupby(['release_year']).revenue.sum().sort_values(ascending = False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [29]:
answers['20'] = '2014'
#+

data[data.production_companies.str.contains('Warner Bros.', na=False)].groupby(['release_year']).profit.sum().sort_values(ascending = False)

release_year
2014    2295464519
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1855493377
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1343545668
2012    1258020056
2002    1022709901
2015     870368348
2006     620170743
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [30]:
answers['21'] = 'Сентябрь'
#+

month = ['','Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
data21 = data.copy()
data21['month'] = data21.release_date.apply(lambda x: month[int(x.split('/')[0])])
data21.groupby(['month'])['imdb_id'].count().sort_values(ascending=False)

month
Сентябрь    227
Декабрь     190
Октябрь     186
Август      161
Март        156
Апрель      149
Июнь        147
Ноябрь      146
Июль        142
Май         140
Февраль     135
Январь      110
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [31]:
answers['22'] = '450'
#+

data22 = data.copy()
data22[(data22.month == 'Июнь') | (data22.month == 'Июль') | (data22.month == 'Август')].imdb_id.count()

AttributeError: 'DataFrame' object has no attribute 'month'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [ ]:
answers['23'] = 'Steven Soderbergh'
#

data23 = data.copy()
data23[(data23.month == 'Декабрь') | (data23.month == 'Январь') | (data23.month == 'Февраль')].imdb_id.count()

director_count_dict = {}
director_count_dict = counter_dict(data23, director_count_dict, 'director')
sorted(director_count_dict.items(), reverse=True, key=lambda x: x[1])[0]

('Steven Soderbergh', 13)

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [ ]:
answers['24'] = 'Four By Two Productions'
#

data24 = data.copy()
data24['title_len'] = data24.original_title.str.len()

data24['production_companies'] = data24.production_companies.str.split('|')
data24.explode('production_companies').groupby(['production_companies'])['title_len'].mean().sort_values(ascending=False)

production_companies
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Polsky Films                  46.0
Museum Canada Productions     46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
Global Entertainment Group     2.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Name: title_len, Length: 1771, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [ ]:
answers['25'] = 'Midnight Picture Show'
#

data25 = data.copy()
data25['overview'] = data25.overview.str.len()

data25['production_companies'] = data25.production_companies.str.split('|')
data25.explode('production_companies').groupby(['production_companies'])['overview'].mean().sort_values(ascending=False)

production_companies
Midnight Picture Show                    1000.0
Room 9 Entertainment                      964.0
Brookwell-McNamara Entertainment          936.0
Lions Gate Family Entertainment           909.0
Crest Animation Productions               909.0
                                          ...  
London Boulevard                           74.0
Henceforth                                 74.0
Phantom Four                               72.0
Empire Pictures                            62.0
Motion Picture Corporation of America      59.0
Name: overview, Length: 1771, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [ ]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
#
data.sort_values('vote_average', ascending=False)['original_title'].head(10)


599                                   The Dark Knight
118                                      Interstellar
125                                The Imitation Game
9                                          Inside Out
34                                               Room
1183                          The Wolf of Wall Street
128                                         Gone Girl
1191                                 12 Years a Slave
119                           Guardians of the Galaxy
1081    The Lord of the Rings: The Return of the King
Name: original_title, dtype: object

In [ ]:
#Более "правильный" способ
data.loc[data.vote_average > data.vote_average.quantile(0.99)].sort_values('vote_average', ascending=False)['original_title']

599                                   The Dark Knight
9                                          Inside Out
34                                               Room
118                                      Interstellar
125                                The Imitation Game
119                           Guardians of the Galaxy
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [32]:

data27 = data.copy()
data27['cast'] = data27.cast.str.split('|')
data27.cast = data27.cast.apply(lambda x: list(combinations(sorted(x), 2)))
data27.explode('cast').cast.value_counts()

(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Emma Watson)        8
(Daniel Radcliffe, Rupert Grint)       8
(Helena Bonham Carter, Johnny Depp)    6
(Ben Stiller, Owen Wilson)             6
                                      ..
(Ewan McGregor, Halle Berry)           1
(Ahney Her, Brian Haley)               1
(Beth Grant, Brian Cox)                1
(Gong Li, Youki Kudoh)                 1
(Danny Huston, Josh Lucas)             1
Name: cast, Length: 17943, dtype: int64

# Submission

In [ ]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

In [ ]:
# и убедиться что ни чего не пропустил)
len(answers)